In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters= 250

cuda


In [2]:
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of available GPUs: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")

Number of available GPUs: 1
GPU 0: NVIDIA GeForce RTX 4060 Laptop GPU


In [3]:
with open('wiz_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = (len(chars))


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [4]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:200])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50,  9,  1, 39, 50, 37, 25,  1, 39,
        30,  1, 39, 50,  9,  1, 29, 44, 27, 11,  0,  0,  1,  1, 33, 36, 36, 45,
        43, 44, 42, 25, 44, 29, 28,  1, 26, 49,  1, 34, 39, 32, 38,  1, 42, 11,
         1, 38, 29, 33, 36, 36,  0,  0,  1,  1, 26, 39, 39, 35, 43,  1, 39, 30,
         1, 47, 39, 38, 28, 29, 42,  1, 47, 33, 36, 36, 33, 25, 37,  1, 37, 39,
        42, 42, 39, 47,  1,  4,  1, 27, 39, 11,  9,  1, 33, 38, 27, 11,  1, 38,
        29, 47])


In [5]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x= torch.stack([data[i:i+block_size]for i in ix])
    y= torch.stack([data[i+1:i+block_size+1]for i in ix])
    x,y= x.to(device), y.to(device)
    return x,y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[73, 68,  1, 58, 54, 73,  1, 78],
        [72,  1, 76, 62, 73, 61,  1, 61],
        [74, 73,  1, 28, 68, 71, 68, 73],
        [67, 73, 58, 57,  1, 78, 68, 74]], device='cuda:0')
targets:
tensor([[68,  1, 58, 54, 73,  1, 78, 68],
        [ 1, 76, 62, 73, 61,  1, 61, 62],
        [73,  1, 28, 68, 71, 68, 73, 61],
        [73, 58, 57,  1, 78, 68, 74,  9]], device='cuda:0')


In [6]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k]= loss.item()
        out[split] =losses.mean()
    return out

In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets= None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T ,C= logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits= logits[:, -1, :]
            probs = F.softmax(logits, dim =-1)
            index_next  = torch.multinomial(probs, num_samples = 1 )
            index = torch.cat((index,index_next), dim = 1)
        return index
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype= torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens= 500)[0].tolist())
print(generated_chars)


8J4nbpFKQvIGeeV]fV;HNUhU(b!fl]V5kr*((8v]ZqVCoM*﻿rn[﻿l
WE1t[﻿rKyQ﻿[b343r'Opi'l2IGXPopeIyOLzXB 3Vmwgm0)'3!E 0Y5;5W﻿IX*Z,Kg05gmcN﻿)JaUNbh_MEol-q-* MgYKM4T]i
-Kg(Yf6r,]ARD1PV﻿mcqS!1Wm8oUM51
?X﻿l
2FjIg(]4'0qw_:[n 87H7ixFKiud lR_v."H7n!l
&S:Y:PRE5mfBwO]tWV )i(E!3(M8O05*!,"lKDcm1HT5J2uH92tHn4[NpuBhogc)CbObPwT
R9vpS2i[x[m2.GU5QNx[.['8&
KHwr-BpccPXBxgYeZ'P_z]eEtHEnJ'Cu'Mg0ZJpCAQlrPWl51[bwxW"!8h']RElKmfU]MdCMqrdi97WA67HTIHzu:vVid*7,f1(,A8Op&XB3p h4]05QmbOWbl5BxFn*ZIypGAlBwuy2";K4gBy[s6Nhkk:c"eudzh*OZ?-(ED


In [8]:
# optimizer pytorch
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step:{iter}, train loss : {losses['train']}, val loss: {losses['val']}")
    xb,yb = get_batch('train')

    #evaluate the loss 
    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print(loss.item())

step:0, train loss : 4.87032413482666, val loss: 4.859932899475098
step:250, train loss : 4.830166339874268, val loss: 4.785061836242676
step:500, train loss : 4.751989841461182, val loss: 4.723567962646484
step:750, train loss : 4.684187412261963, val loss: 4.690806865692139
4.670122146606445


In [9]:
context = torch.zeros((1,1), dtype= torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens= 500)[0].tolist())
print(generated_chars)


p*mg7e71GIBWV-*1)gN?22mP6P]gUm"lm2Or!:6OJ(SA51,R!F!)H:e&]T!Fdckf]a!Fdw_!?7woJ.lKQ8bEcy,.UGvGR6c_foi'j,.jsML!O6]y)cxjsfXBQ3V5gREt'- J
QIMg;fAmmwxS)-kLHFWuRksWv.uBIFpqxO*eEV5KBN?p eDc EKOr﻿74CFoHy:YXBpc&zRs "WL5(6k5';5RIyYZkKR!FXpudBRlUy3Y﻿-Fm;mmff(,QvKuvc;C"Aod[s9vp﻿lEbSpK(3A,7Mg[)H7
!]Z54z-sWS:iM4o-WLsGGUNjT?ou1dv7C46Mk﻿lK]0)&L?pa7)IJ
;ZKnQ;YwamPL-3p
 jslKvZKQ4'?wsTwvr"Q﻿DA5Q;8C"ysoRLp
&s4﻿7zi8CSaI-9XIfN?2&.W1PXPs'4RsDc1dV]HyO"FCGeEksY﻿U﻿aeFYaHrclDcs4RnFtxoPD-_fT-KM,BzN1zg
')2r,TCJpu2Lp;52H,oQ;M
